from IPython.core.display import display, HTML


## Estimating IR flux from IR luminosity
*made by Y. Fudamoto*

Currently two methods are available in this code:
- Bethermin+15 model
- Optically thin modified blackbody

For both methods, conversion factors are pre-calculated and stored in pickles.
Please download the pickles from <a href="https://www.dropbox.com/sh/d6mq8j8ovu840e5/AABFm9KeV5Qkv5LTiKwbDNZBa?dl=0">here</a>.
***

### Bethermin+15 Model
First, you need to load libraries and a pre-calculated pickle file.

In [ ]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import constants
import pickle
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)

Jy2cgs = 1.0e-23

insav_Bethermin =   './SED_pickles/Bethermin15_grid.pickle'
with open(insav_Bethermin,'rb') as handle:
    sed_B15_dict = pickle.load(handle)

__INPUTS__<br>

redshift = redshift<br>
U = dimension less average radiation field. See original paper (U=50 is for ALPINE at z~4-6, normal star-forming galaxies).<br>
rest_wavelength = rest-frame wavelength in __μm__ of the flux you want (e.g., 158 for \[CII\] observations.)<br>
IR_luminosity = IR luminosity in L_sun

__Output__<br>
flux in mJy

In [1]:
def flux_Bethermin(redshift, U, rest_wavelength, IR_luminosity):
    wavelength_sed   = sed_B15_dict['lambda']
    Umean_sed        = sed_B15_dict['Umean']
    nuLnu_sed        =   sed_B15_dict['nuLnu_MS_arr']

    # Select SED depending on Umean
    arg  = np.where(Umean_sed == U)[0][0]

    nuLnu_select = nuLnu_sed[arg]

    arg_wl = np.min(np.where(wavelength_sed >= rest_wavelength))

    # This is "nuLnu/LIR"
    factor  =   nuLnu_select[arg_wl] 
    
    # --- Followings calculte fnu [mJy] from input LIR, factor, redshift, wavelength
    nu_obs = constants.c.cgs.value / ( rest_wavelength * 1.0e-4 * (1.0 + redshift))
    nuLnu = IR_luminosity  * constants.L_sun.cgs.value * factor
    Lnu = nuLnu / nu_obs
    fnu = Lnu / (4 * np.pi * (cosmo.luminosity_distance(redshift).cgs.value)**2. )
    fnu_mJy = fnu/Jy2cgs * 1.0e+3

    return np.array(fnu_mJy)

__Example__
This is for the DEIMOS_COSMOS_873756, first one listed in the table B.1 of <a href="https://ui.adsabs.harvard.edu/abs/2020A%26A...643A...2B/abstract">Bethermin+2020</a>.



In [2]:
flux= flux_Bethermin(4.5457,50,158,10**12.26)
print(flux)

1.2871783865225892


Original measured flux of DEIMOS_COSMOS_873756 is 1.355mJy. So my code gives 5% smaller value. <br>
The difference seems to come from various small differences: mostly from different wavelength, ALPINE's continuum is slightly off from 158um. <br>
But, 5% should be ok enough.
***

### Modified Blackbody
For modified black body, I currently prepared pre-calculated conversion factors for dust temperature ranging 20K to 80K using three different β values (dust emissivity index).<br>
Pickles are separeted based on beta used. So, please choose pickle for beta you want.<br>
I just use beta=1.8 for default.


First, you need to load libraries and a pre-calculated pickle file.
__Choose pickle file depending on beta you want to use__

In [5]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import constants
import pickle
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)


Jy2cgs = 1.0e-23

insav_MBB =   './SED_pickles/MBB_grid_beta1.8.pickle'
with open(insav_MBB,'rb') as handle:
    sed_finegrid_dict = pickle.load(handle)

__INPUTS__<br>

redshift = redshift<br>
dust_T = Dust temperature you want to use: ranging from 20K to 80K.<br>
rest_wavelength = rest-frame wavelength in __μm__ of the flux you want (e.g., 158 for \[CII\] observations.)<br>
IR_luminosity = IR luminosity in L_sun

__Outout__<br>
flux in mJy

In [6]:
def flux_MBB(redshift, dust_T, rest_wavelength, IR_luminosity):
    wavelength_sed   = sed_finegrid_dict['lambda']
    dust_T_sed        = sed_finegrid_dict['dust_T']
    nuLnu_sed        =   sed_finegrid_dict['nuLnu_Lsun']

    # Select SED depending on Umean
    arg  = np.where(dust_T_sed == dust_T)[0][0]

    nuLnu_select = nuLnu_sed[:,arg]

    arg_wl = np.min(np.where(wavelength_sed >= rest_wavelength))

    # This is "nuLnu/LIR"
    factor  =   nuLnu_select[arg_wl] 

    # --- Followings calculte fnu [mJy] from input LIR, factor, redshift, wavelength
    nu_obs = constants.c.cgs.value / ( rest_wavelength * 1.0e-4 * (1.0 + redshift))
    nuLnu = IR_luminosity  * constants.L_sun.cgs.value * factor
    Lnu = nuLnu / nu_obs
    fnu = Lnu / (4 * np.pi * (cosmo.luminosity_distance(redshift).cgs.value)**2. )
    fnu_mJy = fnu/Jy2cgs * 1.0e+3

    return np.array(fnu_mJy)

__Example__

In [7]:
flux= flux_Bethermin(4.5457,45,158,10**12.26)
print(flux)

1.344486229093409
